In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

import utils

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
varname='pr'
varlev='sfc'
dataset='CPC-PRECIP'

basepath='/shared/subx/'
modelpath=basepath+'hindcast/'+varname+varlev+'/daily/anoms/'
verifpath=basepath+'verif/prsfc/daily/anoms/'+dataset+'/'
file_stub='_????01*.anoms.daily'
ofname='all_models_accperfect_CONUS_giorgi.1-12.nc'

gregions=['ENA'] #,'CNA','WNA']
gregnums=[5] #,6,7]

In [4]:
#groups=['ECCC','ESRL','NCEP','RSMAS',
#        'EMC','GMAO','NRL','CESM','CESM']
#
#models=['GEM','FIMr1p1','CFSv2','CCSM4',
#       'GEFS','GEOS_V2p1','NESM','46LCESM1',
#       '30LCESM1']

nenss=[3]
groups=['ECCC']
models=['GEM']

In [5]:
ds_gmask=utils.getGiorgiMask()

In [6]:
all_list=[]

for i,(group,model) in enumerate(zip(groups,models)):

    print(group,model)
    
    # Get Giorgi Regions for SubX Models
    ds_giorgi_emean=utils.getSubXModelGiorgi(modelpath,model,
                                             group,varname,varlev,
                                             file_stub,ds_gmask,
                                             gregions,gregnums)
    
    # Loop over ensemble members and combine
    ens_list=[]
    for iens in np.arange(nenss[i]):
        file_stub_emem=file_stub+'.e'+str(iens+1)
        emem_tmp=utils.getSubXModelGiorgi(modelpath,model,
                                          group,varname,varlev,
                                          file_stub_emem,ds_gmask,
                                          gregions,gregnums)
        ens_list.append(emem_tmp)
        
    ds_giorgi_emean=xr.combine_nested(ens_list,concat_dim='ens')

ds_giorgi_emean
    
    # Calculate perfect model skill
    #skill_tmp=utils.calcPerfectSkill(ds_giorgi_emean,ds_giorgi_emem)
    
    # Combine the sliding skill together into a dimension
    #tmp=xr.combine_nested(skill_tmp,concat_dim='swin').compute()
    
    # Add the sliding window skill to the giant list for this model
    #all_list.append(tmp)

ECCC GEM
/shared/subx/hindcast/prsfc/daily/anoms/ECCC-GEM/pr_sfc_ECCC-GEM_????01*.anoms.daily.nc
/shared/subx/hindcast/prsfc/daily/anoms/ECCC-GEM/pr_sfc_ECCC-GEM_????01*.anoms.daily.e1.nc
/shared/subx/hindcast/prsfc/daily/anoms/ECCC-GEM/pr_sfc_ECCC-GEM_????01*.anoms.daily.e2.nc
/shared/subx/hindcast/prsfc/daily/anoms/ECCC-GEM/pr_sfc_ECCC-GEM_????01*.anoms.daily.e3.nc


<xarray.Dataset>
Dimensions:  (ens: 3, ic: 496, time: 32)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 22 23 24 25 26 27 28 29 30 31
  * ic       (ic) datetime64[ns] 1999-01-01 1999-01-02 ... 2014-01-30 2014-01-31
Dimensions without coordinates: ens
Data variables:
    ENA      (ens, ic, time) float32 dask.array<chunksize=(1, 1, 32), meta=np.ndarray>

In [7]:
all_models=xr.combine_nested(all_list,concat_dim='model')
all_models=all_models.persist()
all_models['model']=models
all_models['swin']=smoothlist
all_models

NameError: name 'smoothlist' is not defined

In [ ]:
#figname='all_models_accperfect_CONUS_giorgi.1-12.png'
figname='testperfect.png'
utils.plotSlidingCorrs(all_models,gregions,smoothcolors,
                       smoothlist,figname)